In [51]:
import functools
import numpy as np
import tensorflow as tf

In [52]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

In [53]:
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

In [54]:
!head {train_file_path}

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n
0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
0,male,2.0,3,1,21.075,Third,unknown,Southampton,n
1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
1,female,4.0,1,1,16.7,Third,G,Southampton,n


In [55]:
!head {test_file_path}

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,35.0,0,0,8.05,Third,unknown,Southampton,y
0,male,54.0,0,0,51.8625,First,E,Southampton,y
1,female,58.0,0,0,26.55,First,C,Southampton,y
1,female,55.0,0,0,16.0,Second,unknown,Southampton,y
1,male,34.0,0,0,13.0,Second,D,Southampton,y
1,female,15.0,0,0,8.0292,Third,unknown,Queenstown,y
0,female,8.0,3,1,21.075,Third,unknown,Southampton,n
0,male,21.0,0,0,8.05,Third,unknown,Southampton,y
0,female,18.0,2,0,18.0,Third,unknown,Southampton,n


In [57]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [58]:
def get_dataset(file_path, **kwargs):
    dataset = tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size=5, # Artificially small to make examples easier to show.  batch_size 
        label_name=LABEL_COLUMN,
        na_value="?",
        num_epochs=1,
        ignore_errors=True, 
        **kwargs)
    return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [61]:
def show_batch(dataset):
    for batch, label in dataset.take(1):
        for key, value in batch.items():
            print("{:20s}: {}".format(key,value.numpy()))

In [62]:
show_batch(raw_train_data)

sex                 : [b'female' b'female' b'female' b'female' b'female']
age                 : [28. 28. 28. 28. 26.]
n_siblings_spouses  : [1 1 0 1 1]
parch               : [0 0 0 0 1]
fare                : [ 15.5    51.862  13.    133.65   26.   ]
class               : [b'Third' b'First' b'Second' b'First' b'Second']
deck                : [b'unknown' b'D' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Queenstown' b'Southampton' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'n' b'n' b'y' b'n' b'n']


In [63]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']
temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)
show_batch(temp_dataset)

sex                 : [b'male' b'male' b'female' b'male' b'male']
age                 : [40.5 28.  35.  33.  35. ]
n_siblings_spouses  : [0 0 1 0 0]
parch               : [0 0 0 0 0]
fare                : [ 7.75   7.225 83.475 12.275  7.05 ]
class               : [b'Third' b'Third' b'First' b'Second' b'Third']
deck                : [b'unknown' b'unknown' b'C' b'unknown' b'unknown']
embark_town         : [b'Queenstown' b'Cherbourg' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'y' b'y' b'n' b'y' b'y']


In [64]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 : [28. 30. 28. 14. 29.]
n_siblings_spouses  : [0 1 3 1 1]
class               : [b'First' b'Third' b'Third' b'Second' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
alone               : [b'y' b'n' b'n' b'n' b'n']


In [65]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [47. 46. 33. 28. 25.]
n_siblings_spouses  : [0. 0. 0. 0. 0.]
parch               : [0. 0. 0. 0. 0.]
fare                : [52.  79.2  9.5 30.5 13. ]


In [66]:
example_batch, labels_batch = next(iter(temp_dataset)) 

In [24]:
def pack(features, label):
      return tf.stack(list(features.values()), axis=-1), labels_batch

In [67]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
    print(features.numpy())
    print()
    print(labels.numpy())

[[ 35.      1.      1.     20.25 ]
 [ 63.      0.      0.      9.587]
 [ 24.      0.      0.      7.496]
 [ 24.      0.      0.     13.   ]
 [ 25.      1.      2.    151.55 ]]

[0 0 0 0 1]


In [68]:
show_batch(raw_train_data)

sex                 : [b'female' b'male' b'male' b'male' b'male']
age                 : [39. 30. 45. 36. 36.]
n_siblings_spouses  : [1 0 0 0 0]
parch               : [1 0 0 0 0]
fare                : [79.65  10.5    6.975  0.     7.496]
class               : [b'First' b'Second' b'Third' b'Third' b'Third']
deck                : [b'E' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'n' b'y' b'y' b'y' b'y']


In [69]:
example_batch, labels_batch = next(iter(temp_dataset)) 

In [70]:
class PackNumericFeatures(object):
    def __init__(self, names):
        self.names = names
    def __call__(self, features, labels):
        numeric_features = [features.pop(name) for name in self.names]
        numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
        numeric_features = tf.stack(numeric_features, axis=-1)
        features['numeric'] = numeric_features

        return features, labels

In [71]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [72]:
show_batch(packed_train_data)

sex                 : [b'male' b'male' b'female' b'male' b'male']
class               : [b'First' b'Third' b'First' b'Third' b'Third']
deck                : [b'A' b'unknown' b'B' b'unknown' b'unknown']
embark_town         : [b'Cherbourg' b'Southampton' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'y' b'y' b'n' b'y' b'y']
numeric             : [[71.     0.     0.    34.654]
 [28.     0.     0.     9.5  ]
 [36.     0.     2.    71.   ]
 [34.     0.     0.     6.496]
 [32.     0.     0.     7.925]]


In [73]:
example_batch, labels_batch = next(iter(packed_train_data)) 

In [74]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc 

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [34]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [35]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

In [75]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x182a72e1e0>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [76]:
example_batch['numeric']

<tf.Tensor: id=12175, shape=(5, 4), dtype=float32, numpy=
array([[21.   ,  0.   ,  0.   ,  8.05 ],
       [28.   ,  0.   ,  0.   ,  7.787],
       [24.   ,  0.   ,  0.   ,  7.142],
       [33.   ,  1.   ,  0.   , 53.1  ],
       [28.   ,  0.   ,  0.   ,  7.75 ]], dtype=float32)>

In [77]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-0.69 , -0.474, -0.479, -0.482],
       [-0.13 , -0.474, -0.479, -0.487],
       [-0.45 , -0.474, -0.479, -0.499],
       [ 0.269,  0.395, -0.479,  0.343],
       [-0.13 , -0.474, -0.479, -0.488]], dtype=float32)

In [79]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}


In [80]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(key=feature, 
                                                                        vocabulary_list=vocab)
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [81]:
# See what you just created.
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [82]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


In [83]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [84]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 1.     0.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.    -0.69  -0.474
 -0.479 -0.482  1.     0.   ]


In [85]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1),
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

In [86]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [87]:
model.fit(train_data, epochs=20)

Epoch 1/20
126/126 [==============================] - 3s 25ms/step - loss: 0.4904 - accuracy: 0.7496
Epoch 2/20
126/126 [==============================] - 1s 6ms/step - loss: 0.4238 - accuracy: 0.8262
Epoch 3/20
126/126 [==============================] - 1s 4ms/step - loss: 0.4021 - accuracy: 0.8262
Epoch 4/20
126/126 [==============================] - 0s 4ms/step - loss: 0.3849 - accuracy: 0.8262
Epoch 5/20
126/126 [==============================] - 0s 4ms/step - loss: 0.3843 - accuracy: 0.8373
Epoch 6/20
126/126 [==============================] - 0s 4ms/step - loss: 0.3731 - accuracy: 0.8341
Epoch 7/20
126/126 [==============================] - 0s 4ms/step - loss: 0.3706 - accuracy: 0.8485
Epoch 8/20
126/126 [==============================] - 0s 4ms/step - loss: 0.3556 - accuracy: 0.8533
Epoch 9/20
126/126 [==============================] - 0s 4ms/step - loss: 0.3535 - accuracy: 0.8485
Epoch 10/20
126/126 [==============================] - 0s 4ms/step - loss: 0.3441 - accuracy: 0.846

In [88]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

53/53 [==============================] - 1s 17ms/step - loss: 0.4517 - accuracy: 0.8598.4765 


Test Loss 0.45165292068191293, Test Accuracy 0.8598484992980957


In [89]:
predictions = model.predict(test_data)

# Show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
    prediction = tf.sigmoid(prediction).numpy()
    print("Predicted survival: {:.2%}".format(prediction[0]),
          " | Actual outcome: ",
          ("SURVIVED" if bool(survived) else "DIED"))


Predicted survival: 51.73%  | Actual outcome:  DIED
Predicted survival: 99.98%  | Actual outcome:  SURVIVED
Predicted survival: 66.15%  | Actual outcome:  SURVIVED
Predicted survival: 6.60%  | Actual outcome:  SURVIVED
Predicted survival: 98.98%  | Actual outcome:  SURVIVED
